In [37]:
import pandas as pd

In [109]:
print('Atividade 2')

# Carrega os dados de full, renomeia a coluna 'rodata' e imprimi um prévia dos dados
full_dataset_path = 'data/campeonato-brasileiro-full.csv'
df_full = pd.read_csv(full_dataset_path)
df_full = df_full.rename(columns={'rodata':'rodada'})
print('DF: Full')
display(df_full.head())

# Carrega os dados de stats, remove a coluna 'rodata' e imprimi um prévia dos dados
stats_dataset_path = 'data/campeonato-brasileiro-estatisticas-full.csv'
df_stats = pd.read_csv(stats_dataset_path)
df_stats = df_stats.drop(columns=['rodata'])
print('DF: Stats')
display(df_stats.head())

# Cria um df auxiliar, altera os nomes das colunas, insere as stats do time mandante e apaga o df auxiliar da memória
df_stats_mandante = df_stats.copy()
df_stats_mandante.columns = [col + "_mandante" for col in df_stats_mandante.columns]
df_main = pd.merge(df_full, df_stats_mandante, how = 'inner', left_on = ['ID', 'mandante'], right_on = ['partida_id_mandante','clube_mandante'])
df_main = df_main.drop(columns=['partida_id_mandante', 'clube_mandante'])
del df_stats_mandante

# Cria um df auxiliar, altera os nomes das colunas, insere as stats do time visitante e apaga o df auxiliar da memória
df_stats_visitante = df_stats.copy()
df_stats_visitante.columns = [col + "_visitante" for col in df_stats_visitante.columns]
df_main = pd.merge(df_main, df_stats_visitante, how = 'inner', left_on = ['ID', 'visitante'], right_on = ['partida_id_visitante','clube_visitante'])
df_main = df_main.drop(columns=['partida_id_visitante', 'clube_visitante'])
del df_stats_visitante

print('DF: Main')
display(df_full.head())

print('Atividade 3')
print('Questão 1: A posse de bola influencia no resultado?')

# Troca a tipagem da coluna vencedor para boolean. É true caso o mandante tenha vencido e falso caso contrário.
df_main_posseXvenc = df_main.copy()
df_main_posseXvenc = df_main_posseXvenc[['posse_de_bola_mandante','vencedor']]
df_main_posseXvenc['vencedor'] = df_main.apply(lambda row: True if row['vencedor'] == row['mandante'] else False, axis=1)

# Transforma posse em numérico
df_main_posseXvenc['posse_de_bola_mandante'] = df_main_posseXvenc['posse_de_bola_mandante'].str.replace('%', '')
df_main_posseXvenc['posse_de_bola_mandante'] = pd.to_numeric(df_main_posseXvenc['posse_de_bola_mandante'], errors = 'coerce')

# Remove valores NaN presentes em 'posse_de_bola'. Necessário para a análise de correlação com 'vencedor'
df_main_posseXvenc = df_main_posseXvenc.dropna()

# Gera a correlação de Pearson entre vencedor e posse de bola
correlacao = df_main_posseXvenc['posse_de_bola_mandante'].corr(df_main_posseXvenc['vencedor'])

print(f'Correlação de Pearson: {correlacao.round(2)}' )
print('Conclusão: a correlação entre as duas variáveis é fraca e negativa. Ou seja, por ser uma correlação fraca, podemos \nconcluir que a posse de bola de um time não tem influència sobre a probabilidade desse mesmo time ganhar.')

print('\n\nQuestão 2: A vantagem de jogar em casa (mandante) é real?')
df_mandXvenc = df_main.copy()
df_mandXvenc = df_mandXvenc[['mandante', 'vencedor']]
df_mandXvenc['vencedor'] = df_mandXvenc.apply(lambda row: True if row['vencedor'] == row['mandante'] else False, axis=1)
count =  df_mandXvenc['vencedor'].value_counts().reset_index()
count['pcent'] = ((count['count'] / count['count'].sum())*100).round(1)
count.columns = ['Vencedor é mandante?', 'Count', '%']
print(count)
print('Conclusão: como pode ser visto na tabela acima, a porcentagem de partidas em que os mandantes vencem é de aproximadamente \n50%. Ou seja, pode-se concluir que ser ou não mandante da partida também não tem influência no resultado do jogo.')


Atividade 2
DF: Full


,ID,rodada,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado
0,1,1,29/03/2003,16:00,Guarani,Vasco,NaN,NaN,NaN,NaN,Guarani,Brinco de Ouro,4,2,SP,RJ
1,2,1,29/03/2003,16:00,Athletico-PR,Gremio,NaN,NaN,NaN,NaN,Athletico-PR,Arena da Baixada,2,0,PR,RS
2,3,1,30/03/2003,16:00,Flamengo,Coritiba,NaN,NaN,NaN,NaN,-,Maracanã,1,1,RJ,PR
3,4,1,30/03/2003,16:00,Goias,Paysandu,NaN,NaN,NaN,NaN,-,Serra Dourada,2,2,GO,PA
4,5,1,30/03/2003,16:00,Internacional,Ponte Preta,NaN,NaN,NaN,NaN,-,Beira Rio,1,1,RS,SP


DF: Stats


,partida_id,clube,chutes,chutes_no_alvo,posse_de_bola,passes,precisao_passes,faltas,cartao_amarelo,cartao_vermelho,impedimentos,escanteios
0,1,Vasco,0,0,NaN,0,NaN,0,0,0,0,0
1,1,Guarani,0,0,NaN,0,NaN,0,0,0,0,0
2,2,Athletico-PR,0,0,NaN,0,NaN,0,0,0,0,0
3,2,Gremio,0,0,NaN,0,NaN,0,0,0,0,0
4,3,Flamengo,0,0,NaN,0,NaN,0,0,0,0,0


DF: Main


,ID,rodada,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado
0,1,1,29/03/2003,16:00,Guarani,Vasco,NaN,NaN,NaN,NaN,Guarani,Brinco de Ouro,4,2,SP,RJ
1,2,1,29/03/2003,16:00,Athletico-PR,Gremio,NaN,NaN,NaN,NaN,Athletico-PR,Arena da Baixada,2,0,PR,RS
2,3,1,30/03/2003,16:00,Flamengo,Coritiba,NaN,NaN,NaN,NaN,-,Maracanã,1,1,RJ,PR
3,4,1,30/03/2003,16:00,Goias,Paysandu,NaN,NaN,NaN,NaN,-,Serra Dourada,2,2,GO,PA
4,5,1,30/03/2003,16:00,Internacional,Ponte Preta,NaN,NaN,NaN,NaN,-,Beira Rio,1,1,RS,SP


Atividade 3
Questão 1: A posse de bola influencia no resultado?
Correlação de Pearson: -0.16
Conclusão: a correlação entre as duas variáveis é fraca e negativa. Ou seja, por ser uma correlação fraca, podemos 
concluir que a posse de bola de um time não tem influència sobre a probabilidade desse mesmo time ganhar.


Questão 2: A vantagem de jogar em casa (mandante) é real?
   Vencedor é mandante?  Count     %
0                 False   4426  50.4
1                  True   4359  49.6
Conclusão: como pode ser visto na tabela acima, a porcentagem de partidas em que os mandantes vencem é de aproximadamente 
50%. Ou seja, pode-se concluir que ser ou não mandante da partida também não tem influência no resultado do jogo.


In [ ]:
# gols_dataset_path = 'data/campeonato-brasileiro-gols.csv'
# df_gols = pd.read_csv(gols_dataset_path)
# df_gols = df_gols.drop(columns=['rodata'])
# print('Gols')
# display(df_gols.head())

# cartoes_dataset_path = 'data/campeonato-brasileiro-cartoes.csv'
# df_cartoes = pd.read_csv(cartoes_dataset_path)
# display(df_cartoes.head())